In [46]:
import pandas as pd
import os

In [47]:
#读入package_label.tsv:每个app应用的类别信息
package_label=pd.read_csv('C:/Users/CT/Desktop/Demo/package_label.tsv',sep='\t',header=None,engine='python',encoding='utf-8')
package_label.rename({0:'app_id',1:'app_category',2:'app_use'},axis=1,inplace=True)
package_label.head()

,app_id,app_category,app_use
0,8c4ac9e7f1d13362a7a443d9fe385d48,实用工具,日历应用
1,991a8d9248a0b224c8cdc6feda5c70a9,视频,网络电视
2,b255f4de409d3f905b8432e6564cd243,母婴亲子,孕育工具助手
3,215263e647821ceb627c55a27475871a,金融,投资理财
4,92b700e7f608dbfe4247809dc6e6dfd0,其它,其它


In [48]:
#读入deviceid_train.tsv：每个设备对应的性别、年龄段训练数据
deviceid_train=pd.read_csv('C:/Users/CT/Desktop/Demo/deviceid_train.tsv',sep='\t',header=None,engine='python',encoding='utf-8')
deviceid_train.rename({0:'device_id',1:'sex',2:'age_level'},axis=1,inplace=True)
deviceid_train.head()

,device_id,sex,age_level
0,bd86d59afa24a839ce6029d718accb19,1,3
1,e7d158c9a8262a35c9cc630a15a9103e,1,5
2,97abdc3828448b5acc7428dd307bc635,2,5
3,e4dbdbf07c9cff03d79f4872e65742b4,1,4
4,6bd4537b2970c5c6ab765c1860b88aa5,1,3


In [49]:
#读入deviceid_test.tsv：用于进行模型验证的设备ID数据
deviceid_test=pd.read_csv('C:/Users/CT/Desktop/Demo/deviceid_test.tsv',sep='\t',header=None,engine='python',encoding='utf-8')
deviceid_test.rename({0:'test_id'},axis=1,inplace=True)
deviceid_test.head()

,test_id
0,2e95e7361f04b5c5c0662e1dcb6fbc65
1,42013074fa7d8d11768e6d3c3e58df56
2,9cc488a211a7c355f99c52ed42e34cb1
3,29dea7b5d58e1e2fe1400a69c5ac33f5
4,365c63569569d88ed44b347fd7c308bd


In [50]:
#读入deviceid_packages.tsv:每个设备及其安装的app列表
deviceid_packages=pd.read_csv('C:/Users/CT/Desktop/Demo/deviceid_packages.tsv',sep='\t',header=None,engine='python',encoding='utf-8')
deviceid_packages.rename({0:'device_id',1:'app_id_lst'},axis=1,inplace=True)
deviceid_packages.head()

,device_id,app_id_lst
0,00009270c4ec26e1d76f5d86847009c9,"1896072db9ce6406febfc17f681c2086,90cb852cf345e..."
1,000189ef5d5b951841d416a8c6c5b995,"1896072db9ce6406febfc17f681c2086,97d0422a3317b..."
2,00026d79a6f0955fc860947724e24765,c33b35d6254ad9c0c238233eb97a6c60
3,0002e3afb8146bc08e40575e45f0eca6,"1896072db9ce6406febfc17f681c2086,07e967d75aab2..."
4,0004709a296f9b925ae283efe2f043e7,"4538778ad75aa8ce61c9d13fb9cb661b,86f9f299cdbc8..."


In [51]:
#读入deviceid_brand.tsv:每个设备的品牌和型号
deviceid_brand=pd.read_csv('C:/Users/CT/Desktop/Demo/deviceid_brand.tsv',sep='\t',header=None,engine='python',encoding='utf-8')
deviceid_brand.rename({0:'device_id',1:'device_brand',2:'device_type'},axis=1,inplace=True)
deviceid_brand.head()

,device_id,device_brand,device_type
0,f69cdc2f5018aa6d3d6500976b926c09,samsung,GT-N7100
1,522690089a93278b65025127d937e9bb,samsung,GT-I9507V
2,dcf13c9ec255153bfa909dc3c64908c3,Sony,L39u
3,58b5f0ec136407caf3708a4b05c1894f,Sony,L39u
4,dbca7cb790645b18383f4536e7f1f303,Sony,XM50h


In [52]:
'''
此段处理效率极慢：放弃！！
#读入deviceid_package_start_close.tsv:每个设备上各个应用的打开、关闭行为数据
with open('C:/Users/CT/Desktop/Demo/deviceid_package_start_close.tsv','r',encoding='utf-8') as f:
    deviceid_package_start_close=pd.DataFrame({},columns=['device_id','app_id','open_time','close_time'])
    line=f.readline()
    count=0
    while line:
        count=count+1
        if(count%5000==0):
            print('处理完第{}行'.format(count))
        deviceid_package_start_close.append(line.split('\t'))
        line=f.readline()
    f.close()
deviceid_package_start_close.head()'''

"\n此段处理效率极慢：放弃！！\n#读入deviceid_package_start_close.tsv:每个设备上各个应用的打开、关闭行为数据\nwith open('C:/Users/CT/Desktop/Demo/deviceid_package_start_close.tsv','r',encoding='utf-8') as f:\n    deviceid_package_start_close=pd.DataFrame({},columns=['device_id','app_id','open_time','close_time'])\n    line=f.readline()\n    count=0\n    while line:\n        count=count+1\n        if(count%5000==0):\n            print('处理完第{}行'.format(count))\n        deviceid_package_start_close.append(line.split('\t'))\n        line=f.readline()\n    f.close()\ndeviceid_package_start_close.head()"

In [53]:
#第一步：将device、所有app类型、app操作行为联立分析

In [54]:
from __future__ import print_function,division
from pyspark import SparkConf,SparkContext
from pyspark.sql import SparkSession


#启动spark
spark=SparkSession.builder.master('local').appName('yiguan').getOrCreate()
sc=spark.sparkContext

##读取tsv文件
path='C:/Users/CT/Desktop/Demo/deviceid_package_start_close.tsv'
schema=None
sep='\t'
header=None
csvDF=spark.read.csv(path=path,schema=schema,sep=sep,header=header)
print(type(csvDF))
csvDF.head()
csvDF.show()

<class 'pyspark.sql.dataframe.DataFrame'>
+--------------------+--------------------+-------------+-------------+
|                 _c0|                 _c1|          _c2|          _c3|
+--------------------+--------------------+-------------+-------------+
|e0450666692b72a1f...|1896072db9ce6406f...|1489025123086|1489025133100|
|e0450666692b72a1f...|1896072db9ce6406f...|1489025393513|1489025398515|
|e0450666692b72a1f...|1896072db9ce6406f...|1489025463551|1489026583522|
|e0450666692b72a1f...|1896072db9ce6406f...|1489034464940|1489034484924|
|e0450666692b72a1f...|1896072db9ce6406f...|1489050176275|1489050201276|
|e0450666692b72a1f...|1896072db9ce6406f...|1489128605368|1489128659958|
|e0450666692b72a1f...|1896072db9ce6406f...|1489132832184|1489132997176|
|e0450666692b72a1f...|1896072db9ce6406f...|1489139754235|1489139764311|
|e0450666692b72a1f...|1896072db9ce6406f...|1489146533857|1489146703041|
|e0450666692b72a1f...|1896072db9ce6406f...|1489146919475|1489146965865|
|e0450666692b72a1f...|

In [55]:
#一共多少行数据
csvDF.select('_c0').count()

36720940

In [56]:
#重命名columns
csvDF=csvDF.withColumnRenamed("_c0","device_id").withColumnRenamed("_c1","app_id").withColumnRenamed("_c2","start_time").withColumnRenamed("_c3","close_time")

In [57]:
csvDF.printSchema()

root
 |-- device_id: string (nullable = true)
 |-- app_id: string (nullable = true)
 |-- start_time: string (nullable = true)
 |-- close_time: string (nullable = true)



In [58]:
#注意：对pyspark的dataframe所做的修改是返回一个新的dataframe，要赋值才能对原来的dataframe进行修改
#将毫秒级时间戳转换为日期
from pyspark.sql.functions import from_unixtime,col
#修改start_time列的类型
csvDF.start_time.cast('float')
csvDF=csvDF.withColumn('start_timestamp',col('start_time')/1000.0)
csvDF=csvDF.withColumn('start_time_mm',from_unixtime(csvDF.start_timestamp,format=("yyyy-MM-dd HH:mm:sss")))

#修改close_time列的类型
csvDF.close_time.cast('float')
csvDF=csvDF.withColumn('close_timestamp',col('close_time')/1000.0)
csvDF=csvDF.withColumn('close_time_mm',from_unixtime(csvDF.close_timestamp,format=("yyyy-MM-dd HH:mm:sss")))

In [59]:
csvDF.select(['start_time_mm','close_time_mm']).show(2)

+--------------------+--------------------+
|       start_time_mm|       close_time_mm|
+--------------------+--------------------+
|2017-03-09 10:05:023|2017-03-09 10:05:033|
|2017-03-09 10:09:053|2017-03-09 10:09:058|
+--------------------+--------------------+
only showing top 2 rows



In [60]:
csvDF=csvDF.drop('start_timestamp','close_timestamp')
# csvDF.show(n=1)

In [61]:
csvDF=csvDF.withColumn('time_len',csvDF.close_time-csvDF.start_time).select('device_id','app_id','start_time','close_time','time_len')
# csvDF.show(2)

In [62]:
csvDF.printSchema()

root
 |-- device_id: string (nullable = true)
 |-- app_id: string (nullable = true)
 |-- start_time: string (nullable = true)
 |-- close_time: string (nullable = true)
 |-- time_len: double (nullable = true)



In [63]:
#计算以秒为单位的app使用时长
time_len=csvDF.withColumn('time_seconds',csvDF.time_len/1000.0).select('device_id','app_id','time_seconds')
# time_len.show(4)

In [64]:
#需要统计特征有：
#1、统计某一设备打开最频繁的app_id
#2、统计某一设备打开时间最长的app_id
from pyspark.sql import functions as func


In [65]:
#找出使用最频繁的app类型：
_temp1=time_len.groupBy("device_id",'app_id').agg(func.count('time_seconds'))
_temp2=_temp1.groupBy('device_id').agg(func.max('count(time_seconds)'))
# _temp2.show(5)

In [66]:
# print(_temp2.count())

In [67]:
# print(_temp1.count())

In [68]:
#将_temp2和原来的_temp1进行join操作，得出对应的app_id，即每个device_id使用最频繁的app_id
_temp2=_temp2.withColumnRenamed('device_id','device_id_1')
_temp3=_temp2.join(_temp1,(_temp2.device_id_1==_temp1.device_id)&(_temp2['max(count(time_seconds))']==_temp1['count(time_seconds)']))

In [69]:
# print(_temp3.count())

In [70]:
#根据每个device_id最频发使用的app_id，结合之前的app类型表，找出每个device_id最频繁使用app类型
_temp4=_temp3.select('device_id','app_id')
# print(_temp4.count())

In [71]:
#一部分device使用最频繁的app存在多个
#指定字段去重
_temp4=_temp4.dropDuplicates(["device_id"])
# print(_temp4.count())

In [72]:
#对app类型进行数值化处理（package_label）
cat_label={}
count=0
for category in package_label['app_category'].unique():
    cat_label[category]=count+1
    count=count+1
#将字典映射到_temp4

In [73]:
package_label['app_type_label']=package_label['app_category'].map(cat_label)
df_app=package_label[['app_id','app_type_label']]
#转换为spark的dataframe

In [74]:
#将df_app转换为pyspark.sql.dataframe
df_app_spark=spark.createDataFrame(df_app)
# df_app_spark.show(5)

In [75]:
#将_temp4与df_app_spark进行join操作，找出每个device_id对应的最频繁使用的app类型
df_app_spark=df_app_spark.withColumnRenamed('app_id','app_id_1')
device_applabel=_temp4.join(df_app_spark,_temp4.app_id==df_app_spark.app_id_1,how='left').drop('app_id','app_id_1')
# print('device_applabel大小：'+str(device_applabel.count()))

In [76]:
#查看device_id和机型的关系：以机型为特征

In [ ]:
not_null_device=device_applabel.filter(func.isnull("app_type_label"))
# not_null_device.show()
print(not_null_device.count())

In [ ]:
#算了，为了模型复现，直接用device的app使用时长作为特征吧
time_len=time_len.withColumnRenamed('device_id','device_id_1')
df_timelen=device_applabel.join(time_len,device_applabel.device_id==time_len.device_id_1,how='left').select('device_id','time_seconds')
# df_timelen.show()

In [ ]:
#计算每个device_id的app数目：deviceid_packages(特征1)
deviceid_packages['app_num']=pd.Series([len(i) for i in deviceid_packages['app_id_lst']])
deviceid_packages.head()

In [ ]:
#app的最多使用次数（特征2）
feature_maxUseTime=_temp2.withColumnRenamed('device_id_1','device_id')
# feature_maxUseTime.show(10)

In [ ]:
df_phone=pd.DataFrame({'phone_num':deviceid_brand.groupby('device_brand')['device_id'].count().sort_values(ascending=False)},columns=['phone_num'])
df_phone['phone_cumsum']=df_phone['phone_num'].cumsum()
df_phone.head(10)

In [ ]:
#实际上19类手机就占了整个手机类型的90%：选取手机型号作为特征
df_most_val=df_phone.loc[df_phone['phone_cumsum']<=df_phone['phone_cumsum'][-1]*0.9,:]
df_most_val

In [ ]:
#找出每一个device_id对应的手机型号并将其映射为标签
map_device_phone={}
phone_type=df_most_val.index.tolist()
phone_type.append('其它')
count=0
for i in phone_type:
    count=count+1
    map_device_phone[i]=count
map_device_phone


#将deviceid_brand中手机类型修改为对应的标签(特征3)
for i in range(len(deviceid_brand['device_brand'])):
    if deviceid_brand['device_brand'][i] not in phone_type:
        deviceid_brand['device_brand'][i]='其它'
deviceid_brand['device_brand']=deviceid_brand['device_brand'].map(map_device_phone)
deviceid_brand

In [ ]:
#特征组合
import pyspark.sql.types as typ
spark=SparkSession.builder.master("local").appName('yiguan').getOrCreate()
sc=spark.sparkContext

# schema=typ.StructType([typ.StructField("device_id",typ.StringType(),True
#                                     ),
#                        typ.StructField(
#                                      "sex",typ.IntegerType(),True),
#                        typ.StructField(
#                                      "age_level",typ.IntegerType(),True)
#                       ])
device_train_spark=spark.createDataFrame(deviceid_train)
device_train_spark.show(4)

type(deviceid_train)

#特征1：app数目
deviceid_packages_spark=spark.createDataFrame(deviceid_packages)
deviceid_packages_spark.show(5)
#特征2：最多使用的app次数



#特征3:手机型号
deviceid_brand_spark=spark.createDataFrame(deviceid_brand[['device_id','device_brand']])
deviceid_brand_spark.show(5)

#训练数据:device_train_spark
train_spark=device_train_spark.join(deviceid_packages_spark,on='device_id',how='left').join(feature_maxUseTime,on='device_id',how='left').join(deviceid_brand_spark,on='device_id',how='left')
# train_spark.show(10)

train_spark=train_spark.drop('app_id_lst')
# train_spark.show(3)

In [ ]:
#如何判断device_train_spark中的所有id在deviceid_brand_spark中都能找到？
# device_train_spark.take(1)

In [ ]:
# deviceid_brand_spark.take(1)

In [ ]:
map_label={'10':1,'11':2,'12':3,'13':4,'14':5,'15':6,'16':7,'17':8,'18':9,'19':10,'110':11,
          '20':12,'21':13,'22':14,'23':15,'24':16,'25':17,'26':18,'27':19,'28':20,'29':21,'210':22}

#下面代码没用上，实际上是构建两个模型分开预测sex和age的
multi_label=train_spark.withColumn('label',func.concat(train_spark['sex'],train_spark['age_level']))

#转换为rdd进行计算
rdd_1=sc.parallelize(multi_label.collect(),2)

#多标签替换为单标签
after_label=rdd_1.map(lambda x:x[6]).map(lambda x:map_label[x]).collect()  #list

#将rdd转换为dataframe
from pyspark.sql.functions import lit,udf
from pyspark.sql.types import DataType,StringType

df_after_label=spark.createDataFrame(after_label,schema='int')  #pyspark.sql.dataframe

myFunc_udf=udf(lambda x:map_label[x],StringType())

multi_label=multi_label.withColumn("label_1",myFunc_udf("label"))

#开始构造机器学习模型需要的输入数据：
#1.将多标签多分类问题转换为单标签多分类问题
#2.采用随机森林模型:对每一个target值拟合一个分类器，可以解决多标签多分类问题
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.feature import StandardScaler
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.tree import RandomForest

#将数据标准化
from pyspark.ml import Pipeline

In [ ]:
multi_label=multi_label.withColumn('device_brand_1',func.round('device_brand'))

multi_label=multi_label.withColumn('app_num_int',func.col('app_num').cast(typ.IntegerType()))
multi_label=multi_label.withColumn('time_seconds_int',func.col('max(count(time_seconds))').cast(typ.IntegerType()))
multi_label=multi_label.withColumn('device_brand_int',func.col('device_brand_1').cast(typ.IntegerType()))
multi_label=multi_label.withColumn('sex_int',func.col('sex').cast(typ.IntegerType()))
multi_label=multi_label.withColumn('age_level_int',func.col('age_level').cast(typ.IntegerType()))
multi_label=multi_label.select(['app_num_int','time_seconds_int','device_brand_int','sex_int','age_level_int'])

In [ ]:
#查询是否有空值
multi_label.filter(func.isnull('device_brand_int')).collect()

In [ ]:
#1、特征聚合
import pyspark.ml.feature as ft
import pyspark.ml.classification as cl
import numpy as np

featureCreator1=ft.VectorAssembler(
            inputCols=["app_num_int","time_seconds_int","device_brand_int"],
            outputCol='features'
)
featureCreator2=ft.VectorAssembler(
            inputCols=["app_num_int","time_seconds_int","device_brand_int"],
            outputCol='features'
)
RFClassifier1=cl.RandomForestClassifier(numTrees=3, maxDepth=2,featuresCol='features', labelCol='sex_int', seed=42)
RFClassifier2=cl.RandomForestClassifier(numTrees=3, maxDepth=2,featuresCol='features',labelCol='age_level_int', seed=42)
pipeline1=Pipeline(stages=[
    featureCreator1,
    RFClassifier1
])
pipeline2=Pipeline(stages=[
    featureCreator2,
    RFClassifier2
])

In [ ]:
model1=pipeline1.fit(multi_label.na.drop())
result=model1.transform(multi_label)
model2=pipeline2.fit(multi_label.na.drop())
result2=model2.transform(multi_label)

In [ ]:
# 会报错，因为multi_label中存在device_brand为None的记录
# sex_pre=result.select('prediction').collect()
# sex_pre

In [ ]:
# 会报错，因为multi_label中存在device_brand为None的记录
# age_pre=result2.select('prediction').collect()
# age_pre

In [ ]:
#将device_test数据按构造的特征进行处理，然后倒入模型进行预测
#训练数据:device_train_spark
schema1=typ.StructType([typ.StructField('device_id',typ.StringType(),False)])
device_test_spark=spark.createDataFrame(deviceid_test,schema=schema1)

test_spark=device_test_spark.join(deviceid_packages_spark,on='device_id',how='left').join(feature_maxUseTime,on='device_id',how='left').join(deviceid_brand_spark,on='device_id',how='left')

test_spark=test_spark.drop('app_id_lst')
test_spark.take(5)

#将测试数据集转换成数字格式
test_dataset=test_spark.withColumn('device_brand_1',func.round('device_brand'))

test_dataset=test_dataset.withColumn('app_num_int',func.col('app_num').cast(typ.IntegerType()))
test_dataset=test_dataset.withColumn('time_seconds_int',func.col('max(count(time_seconds))').cast(typ.IntegerType()))
test_dataset=test_dataset.withColumn('device_brand_int',func.col('device_brand_1').cast(typ.IntegerType()))
test_dataset=test_dataset.select(['app_num_int','time_seconds_int','device_brand_int'])

result_sex=model1.transform(test_dataset.dropna())
result_sex.select('probability').collect()